In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer

from imblearn.over_sampling import SMOTE

from sklearn.impute import KNNImputer

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

import statsmodels.api as sm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

In [2]:
def run_logistic_regression_trials(X_train, y_train):
    ## NOTE: this is actually ridge regression, as the default penalty is l2
    ## Tries solvers: liblinear and lbfgs

    df_list = [] # will be used to create reg_results (full list of scores, one row per run)
    reg_mean_scores_df = pd.DataFrame() # mean scores one row per cv run, also has col used to save the models

    idx = 0
    for cvk in [5,10]:
        for solver_name in ['liblinear', 'lbfgs']:
            reg_model = LogisticRegression(solver=solver_name)
            scores_list = cross_val_score(reg_model, X_train, y_train, cv=cvk)

            df_list.append(pd.DataFrame({'score': scores_list, 'cvk':[cvk]*cvk, 'solver':[solver_name]*cvk}))

            new_score_record = pd.DataFrame({'cvk': cvk, 'solver': solver_name, 'init_model': reg_model, 
                                             'mean_score': np.mean(scores_list)}, index=[idx])
            reg_mean_scores_df = pd.concat([reg_mean_scores_df,new_score_record], axis=0)
            idx += 1
    reg_results = pd.concat(df_list)
    
    return reg_mean_scores_df, reg_results

# Load Data

In [14]:
file_name = "data/in-vehicle-coupon-recommendation.csv"
data = pd.read_csv(file_name)

In [4]:
data.shape

(12684, 26)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

## Check for imbalance

### Full dataset
57% Yes  

### Coupon type subsets

```
coupon type             % Yes
-----------------------------
Restaurant(<20)         70.71
Coffee House            49.92
Carry out & Take away   73.55
Bar                     41.00
Restaurant(20-50)       44.10

           TOTAL record count
-----------------------------
Restaurant(<20)          2786
Coffee House             3996
Carry out & Take away    2393
Bar                      2017
Restaurant(20-50)        1492
```

In [6]:
data.Y.value_counts()

1    7210
0    5474
Name: Y, dtype: int64

In [7]:
7210/12684

0.5684326710816777

In [8]:
for coupon_type in data.coupon.unique():
    vc = data.loc[data['coupon'] == coupon_type, 'Y'].value_counts()
    print(f'{coupon_type} :\t\t{100*vc[1]/(vc[0]+vc[1]):.2f}% Yes')


Restaurant(<20) :		70.71% Yes
Coffee House :		49.92% Yes
Carry out & Take away :		73.55% Yes
Bar :		41.00% Yes
Restaurant(20-50) :		44.10% Yes


## How many records do we have for each coupon type?
Note: Numbers are kind of small... especially for the expensive restaurants

In [9]:
data.coupon.value_counts()

Coffee House             3996
Restaurant(<20)          2786
Carry out & Take away    2393
Bar                      2017
Restaurant(20-50)        1492
Name: coupon, dtype: int64

# Cleaning: dropping features

## Feature 'car' is mostly NULL
Only 108/12684 records include this feature, and it's not mentioned in the feature descriptions.

In [10]:
data.car.value_counts()

Scooter and motorcycle                      22
Mazda5                                      22
do not drive                                22
crossover                                   21
Car that is too old to install Onstar :D    21
Name: car, dtype: int64

In [15]:
data = data.drop('car', axis=1)

## Features 'direction_same' and 'direction_opp' are 100% correlated
direction_same = not direction_opp

Drop direction_opp

In [16]:
data = data.drop('direction_opp', axis=1)

## Combine features 'toCoupon_GEQ\*min' into one column

In [18]:
## confirm these are indeed cumulative metrics (and not exlusive bins) -- YES!
data[['toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min']].value_counts()

data.loc[data['toCoupon_GEQ15min'] == 1, 'toCoupon_GEQ5min'].value_counts()
data.loc[data['toCoupon_GEQ25min'] == 1, 'toCoupon_GEQ15min'].value_counts()

toCoupon_GEQ5min  toCoupon_GEQ15min  toCoupon_GEQ25min
1                 1                  0                    5611
                  0                  0                    5562
                  1                  1                    1511
dtype: int64

1    7122
Name: toCoupon_GEQ5min, dtype: int64

1    1511
Name: toCoupon_GEQ15min, dtype: int64

In [33]:
## populate new column minsToCouponDest with values ['5-14', '15-24', '25plus']
data.loc[(data.toCoupon_GEQ25min == 1), 'minsToCouponDest'] = '25plus'
data.loc[(data.toCoupon_GEQ25min == 0) & (data.toCoupon_GEQ15min==1), 'minsToCouponDest'] = '15-24'
data.loc[(data.toCoupon_GEQ25min == 0) & (data.toCoupon_GEQ15min==0), 'minsToCouponDest'] = '5-14'


# Split by coupon type

In [ ]:
Bar_data = data[data['coupon'] == 'Bar']
Bar_data = Bar_data.drop('coupon', axis=1)
Bar_data.to_csv('data/Bar_data.csv', index=False)

CoffeeHouse_data = data[data['coupon'] == 'Coffee House']
CoffeeHouse_data = CoffeeHouse_data.drop('coupon', axis=1)
CoffeeHouse_data.to_csv('data/CoffeeHouse_data.csv', index=False)

CarryAway_data = data[data['coupon'] == 'Carry out & Take away']
CarryAway_data = CarryAway_data.drop('coupon', axis=1)
CarryAway_data.to_csv('data/CarryAway_data.csv', index=False)

RestaurantLessThan20_data = data[data['coupon'] == 'Restaurant(<20)']
RestaurantLessThan20_data = RestaurantLessThan20_data.drop('coupon', axis=1)
RestaurantLessThan20_data.to_csv('data/RestaurantLessThan20_data.csv', index=False)

Restaurant20To50_data = data[data['coupon'] == 'Restaurant(20-50)']
Restaurant20To50_data = Restaurant20To50_data.drop('coupon', axis=1)
Restaurant20To50_data.to_csv('data/Restaurant20To50_data.csv', index=False)


## Number of categories for each feature

In [71]:
print("Binary features\n")
for colname in data.columns.to_list():
    vals = data[colname].unique()
    n = len(vals)
    if n == 2:
        print(f"{colname} {vals[0]}/{vals[1]}")
        
print("\nNum categories, nonbinary features\n")
for colname in data.columns.to_list():
    vals = data[colname].unique()
    n = len(vals)
    if n > 2:
        print(f"{colname}\t{n}")

Binary features

expiration 1d/2h
gender Female/Male
has_children 1/0
toCoupon_GEQ15min 0/1
toCoupon_GEQ25min 0/1
direction_same 0/1
Y 1/0

Num categories, nonbinary features

destination	3
passanger	4
weather	3
temperature	3
time	5
coupon	5
age	8
maritalStatus	5
education	6
occupation	25
income	9
Bar	6
CoffeeHouse	6
CarryAway	6
RestaurantLessThan20	6
Restaurant20To50	6


## Extract binary features

Includes binary values: 'has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_opp'

(these cols are already numeric 0/1 values)

In [32]:
numeric_data = data.select_dtypes('number')
binary_numeric_data = numeric_data.drop('temperature', axis=1)
binary_numeric_data.columns

Index(['has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min',
       'toCoupon_GEQ25min', 'direction_opp', 'Y'],
      dtype='object')

# Encode Categoricals

Note: temperature is one of 3 values: 30, 55, or 80 (essentially: cold, warm, hot)

These probably aren't the exact temperature, but rather some kind of binned values, so it's really categorical.  
Is it ordinal or nominal? 

IDEA: Try nominal, since one isn't better or worse?

In [16]:
categorical_data = data.select_dtypes('object')
categorical_data.columns

Index(['destination', 'passanger', 'weather', 'time', 'coupon', 'expiration',
       'gender', 'age', 'maritalStatus', 'education', 'occupation', 'income',
       'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
       'Restaurant20To50'],
      dtype='object')

## Encoding options

```
Ordinal

'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50'

Nominal (unordered)

'destination', 'passanger', 'weather', 'expiration', 'gender', 'maritalStatus', 'occupation'

Try both?

'time', 'age', 'education', 'income', 'temperature'

```

## Replace missing values with 'unknown'

In [17]:
nan_colnames = categorical_data.columns[categorical_data.isna().any()].tolist()
nan_colnames

['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']

In [18]:
categorical_data.fillna('unknown', inplace=True)

## Ordinal features

In [19]:
ordinal_colnames = ['income', 'time', 'education', 
                   'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
ordinal_data = categorical_data[ordinal_colnames]

ordinal_categories_list = [['Less than $12500', '$12500 - $24999', '$25000 - $37499', '$37500 - $49999',
                            '$50000 - $62499', '$62500 - $74999', '$75000 - $87499', '$87500 - $99999',
                            '$100000 or More'],
                           ['7AM', '10AM', '2PM', '6PM', '10PM'],
                           ['Some High School', 'High School Graduate', 'Some college - no degree',
                            'Associates degree', 'Bachelors degree', 'Graduate degree (Masters or Doctorate)'],
                           ['unknown', 'never', 'less1', '1~3', '4~8', 'gt8'],
                           ['unknown', 'never', 'less1', '1~3', '4~8', 'gt8'],
                           ['unknown', 'never', 'less1', '1~3', '4~8', 'gt8'],
                           ['unknown', 'never', 'less1', '1~3', '4~8', 'gt8'],
                           ['unknown', 'never', 'less1', '1~3', '4~8', 'gt8']]

ordinal_encoder = OrdinalEncoder(categories=ordinal_categories_list)

## run the encoding
ordinal_encoded_data = pd.DataFrame(ordinal_encoder.fit_transform(ordinal_data), columns=ordinal_data.columns)

## Nominal features

In [74]:
nominal_data_w_temp = categorical_data.drop(ordinal_colnames, axis=1)
nominal_data_w_temp['temperature'] = data.temperature.astype(str)

nominal_data_w_temp.columns

encoded_nominal_data_w_temp = pd.get_dummies(nominal_data_w_temp, drop_first=True)

Index(['destination', 'passanger', 'weather', 'coupon', 'expiration', 'gender',
       'age', 'maritalStatus', 'occupation', 'temperature'],
      dtype='object')

In [75]:
nominal_data_w_temp.head()
encoded_nominal_data_w_temp.head()

,destination,passanger,weather,coupon,expiration,gender,age,maritalStatus,occupation,temperature
0,No Urgent Place,Alone,Sunny,Restaurant(<20),1d,Female,21,Unmarried partner,Unemployed,55
1,No Urgent Place,Friend(s),Sunny,Coffee House,2h,Female,21,Unmarried partner,Unemployed,80
2,No Urgent Place,Friend(s),Sunny,Carry out & Take away,2h,Female,21,Unmarried partner,Unemployed,80
3,No Urgent Place,Friend(s),Sunny,Coffee House,2h,Female,21,Unmarried partner,Unemployed,80
4,No Urgent Place,Friend(s),Sunny,Coffee House,1d,Female,21,Unmarried partner,Unemployed,80


,destination_No Urgent Place,destination_Work,passanger_Friend(s),passanger_Kid(s),passanger_Partner,weather_Snowy,weather_Sunny,coupon_Carry out & Take away,coupon_Coffee House,coupon_Restaurant(20-50),...,occupation_Personal Care & Service,occupation_Production Occupations,occupation_Protective Service,occupation_Retired,occupation_Sales & Related,occupation_Student,occupation_Transportation & Material Moving,occupation_Unemployed,temperature_55,temperature_80
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1
4,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1


## Combine all encoded features into one dataframe

In [33]:
recombined_data = pd.concat([binary_numeric_data, ordinal_encoded_data, encoded_nominal_data_w_temp], axis=1)

## Define X/y train/test

In [34]:
## define X and y 
X = recombined_data.drop('Y', axis=1).reset_index(drop=True)
y = recombined_data.Y

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## logistic_regression_trials

In [35]:
reg_mean_scores_df, reg_results = run_logistic_regression_trials(X_train, y_train)
display(reg_mean_scores_df.sort_values('mean_score', ascending=False))

,cvk,solver,init_model,mean_score
2,10,liblinear,LogisticRegression(solver='liblinear'),0.68059696
0,5,liblinear,LogisticRegression(solver='liblinear'),0.67961154
1,5,lbfgs,LogisticRegression(),0.67645747
3,10,lbfgs,LogisticRegression(),0.67586518


## single random forest

In [36]:
clf = RandomForestClassifier(max_depth=5, random_state=42)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=5)
print(f"{np.mean(cross_val_scores)} {np.std(cross_val_scores)}")

0.6784267044116469 0.00801525847001094


In [37]:
roc_auc_scorer = make_scorer(roc_auc_score, average='weighted')
cross_val_scores = cross_val_score(clf, X_train, y_train, scoring=roc_auc_scorer, cv=5)
print(f"(weighted avg) AUC: {np.mean(cross_val_scores)} {np.std(cross_val_scores)}")

(weighted avg) AUC: 0.6432364265038633 0.008869013853778751


# Split by coupon type

In [38]:
encoded_nominal_data_no_coupon = pd.get_dummies(nominal_data_w_temp.drop('coupon', axis=1), drop_first=True)
data_to_split = pd.concat([binary_numeric_data, ordinal_encoded_data, encoded_nominal_data_no_coupon, 
                           nominal_data.coupon], axis=1)

In [39]:
cols_to_keep = data_to_split.columns.to_list()
cols_to_keep.remove('coupon')

In [40]:
Bar_data = data_to_split.loc[data_to_split['coupon'] == 'Bar', cols_to_keep]
CoffeeHouse_data = data_to_split.loc[data_to_split['coupon'] == 'Coffee House', cols_to_keep]
CarryAway_data = data_to_split.loc[data_to_split['coupon'] == 'Carry out & Take away', cols_to_keep]
RestaurantLessThan20_data = data_to_split.loc[data_to_split['coupon'] == 'Restaurant(<20)', cols_to_keep]
Restaurant20To50_data = data_to_split.loc[data_to_split['coupon'] == 'Restaurant(20-50)', cols_to_keep]

In [42]:
Bar_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2017 entries, 9 to 12682
Data columns (total 59 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   has_children                                          2017 non-null   int64  
 1   toCoupon_GEQ5min                                      2017 non-null   int64  
 2   toCoupon_GEQ15min                                     2017 non-null   int64  
 3   toCoupon_GEQ25min                                     2017 non-null   int64  
 4   direction_opp                                         2017 non-null   int64  
 5   Y                                                     2017 non-null   int64  
 6   income                                                2017 non-null   float64
 7   time                                                  2017 non-null   float64
 8   education                                             201